In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


This notebook run using kaggle notebook. First of all, like usual, we have to import pandas and read dataset. As you can see on the line below, we read data test as `data_test` and sample prediction as `sample_prediction_df`. Sample_prediction is important part in kaggle competition as procedure to submit our machine learning model on this competition.

In [2]:
import pandas as pd

sample_prediction_df  = pd.read_csv("../input/jane-street-market-prediction/example_sample_submission.csv",index_col ='ts_id')
data_test= pd.read_csv("../input/jane-street-market-prediction/example_test.csv",index_col ='ts_id')

Due to the limited amount of RAM and how big dataset for train, we cannot use `pd.read_csv` like usual, in this case, i use `dask.dataframe` library. In this notebook library `dask.dataframe` already installed, u have may install it first before import it like i do.

In [3]:
import dask.dataframe as dd

train = dd.read_csv("../input/jane-street-market-prediction/train.csv").set_index('ts_id').compute()
print("Data size:", train.shape)

Data size: (2390491, 137)


As you can see, on this problem we have more than 20 million row and 137 columns. For dataset detail u can see on data description. Let's see if our dataset have missing value on it! on line below, we create function that sort columns with missing value from the most to the least(in percentage).

In [4]:
#the function
def missing(dff):
    print (round((dff.isnull().sum() * 100/ len(dff)),2).sort_values(ascending=False))

#check the data train using our function that we declare before
missing(train)

feature_18    16.55
feature_28    16.55
feature_27    16.55
feature_17    16.55
feature_7     16.45
              ...  
feature_53     0.00
feature_54     0.00
feature_57     0.00
feature_58     0.00
date           0.00
Length: 137, dtype: float64


From `resp` and `weight` column we have to make 1 additional column, namely `action`. Column `action` will be value as 1 if the multiplication of column `weight` and column `resp` is positive and 0 otherwise. Before that, lets check column `resp` and `weight` have missing value or not.

In [5]:
resp_train = train.loc[:,train.columns.str.contains('resp')]
weight_train = train.loc[:,train.columns.str.contains('weight')]
missing(resp_train)
missing(weight_train)

resp      0.0
resp_4    0.0
resp_3    0.0
resp_2    0.0
resp_1    0.0
dtype: float64
weight    0.0
dtype: float64


Because there are no missing value on `resp` and `weight` columns then we can create column `action`.

In [6]:
train = train[train['weight'] != 0]

train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')

In [7]:
#check data train
train

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,action
ts_id,,,,,,,,,,,,,,,,,,,,,
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,0
4,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,...,NaN,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,0
6,0,0.190575,-0.001939,-0.002301,0.001088,0.005963,0.000709,-1,-3.172026,-3.093182,...,NaN,0.336873,4.076447,0.614783,6.622176,0.800618,5.231595,0.361506,3.921714,1
7,0,3.820844,0.017395,0.021361,0.031163,0.036970,0.033473,-1,0.446050,-0.466210,...,NaN,2.101997,4.846202,1.479875,5.261328,2.305066,4.571762,2.201537,4.429745,1
8,0,0.116557,-0.005460,-0.007301,-0.009085,-0.003546,-0.001677,1,-3.172026,-3.093182,...,NaN,1.537913,4.785838,1.637435,6.968002,2.354338,5.825499,1.778029,4.740577,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390444,499,56.694795,0.001607,0.001607,-0.001245,-0.012068,-0.010023,-1,1.538675,2.530447,...,-2.084489,-0.984942,1.129901,-1.632432,-2.169964,-2.371293,-0.889212,-1.554352,0.215984,0
2390446,499,1.650055,0.004523,0.004523,0.003172,-0.013886,-0.013637,1,0.270380,-1.231874,...,-1.982950,1.724863,-0.984278,1.413825,-1.598825,2.087731,-1.126050,1.590538,-1.250209,0
2390478,499,0.895142,0.000486,0.000486,-0.004090,-0.008105,-0.005441,-1,-0.134380,0.160580,...,-2.103206,-0.765664,-2.148415,-0.599358,-3.155134,-0.957971,-2.285314,-0.894580,-2.064227,0


Furthermore, in preparing the data, we define the column to be trained and the prediction target. In this case i choose `feature` columns to train and `action` as prediction target.

In [19]:
X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = train.loc[:, 'action']

In [20]:
#check X_train
X_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129
ts_id,,,,,,,,,,,,,,,,,,,,,
1,-1,-1.349537,-1.704709,0.068058,0.028432,0.193794,0.138212,NaN,NaN,-0.151877,...,NaN,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684
4,1,-3.172026,-3.093182,-0.161518,-0.128149,-0.195006,-0.143780,NaN,NaN,2.683018,...,NaN,NaN,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633
6,-1,-3.172026,-3.093182,-0.030588,-0.043175,0.097058,0.053483,NaN,NaN,-6.299415,...,NaN,NaN,0.336873,4.076447,0.614783,6.622176,0.800618,5.231595,0.361506,3.921714
7,-1,0.446050,-0.466210,0.498751,0.244116,0.412528,0.224140,NaN,NaN,0.277257,...,NaN,NaN,2.101997,4.846202,1.479875,5.261328,2.305066,4.571762,2.201537,4.429745
8,1,-3.172026,-3.093182,-0.363836,-0.291496,0.128422,0.096168,NaN,NaN,-3.727364,...,NaN,NaN,1.537913,4.785838,1.637435,6.968002,2.354338,5.825499,1.778029,4.740577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390444,-1,1.538675,2.530447,2.494852,3.263345,1.613620,2.097220,-0.401539,-0.489412,-0.045341,...,-1.872247,-2.084489,-0.984942,1.129901,-1.632432,-2.169964,-2.371293,-0.889212,-1.554352,0.215984
2390446,1,0.270380,-1.231874,-5.802676,-3.172423,-4.357278,-2.301009,1.957683,1.000846,4.245754,...,1.210442,-1.982950,1.724863,-0.984278,1.413825,-1.598825,2.087731,-1.126050,1.590538,-1.250209
2390478,-1,-0.134380,0.160580,1.292513,1.453954,0.605912,0.687598,-0.489143,-0.593642,-0.915392,...,-0.342937,-2.103206,-0.765664,-2.148415,-0.599358,-3.155134,-0.957971,-2.285314,-0.894580,-2.064227


In [21]:
#check y
y_train

ts_id
1          0
4          0
6          1
7          1
8          0
          ..
2390444    0
2390446    0
2390478    0
2390481    0
2390489    0
Name: action, Length: 1981287, dtype: int64

About missing value in column `feature` that we check before, we will handle it later. So, we are done for data preparing, next step is build the model.

in this case I use *eXtreme Gradient Boosting* (`xgboost`) or more specifically the `XGBClassifier`

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

After importing `XGBClassifier`, next step is build the model, on the line below you could see the parameter that have been set up. Usually i use `GridSearchCV` to tuning parameter but because RAM limitation in this notebook, what i did is try and error until i got the best prediction.

About missing value that we have check before, i fill it with 0.

In [ ]:
# fit model no training data
model = xgb.XGBClassifier(
    n_estimators=480,
    max_depth=10,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    missing=0, #handle the missing value on feature
    gamma = 0.4,
    min_child_weight=3,
    random_state=0,
    tree_method='gpu_hist' #to activate the GPU on kaggle notebook
)

Fitting the data!

In [ ]:
model.fit(X_train, y_train)

The last line below is to predict the unsee data test. Make sure turn off the internet to run this line below (competition requirement)

In [ ]:
import janestreet
# initiation of the environment
env = janestreet.make_env()
# an iterator to loops over the test set
iter_test = env.iter_test() 

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    #We will specify the X_test from our test data (features)
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
    #Replace the missing value with -999
    X_test.fillna(0)
    #Predict using our X_test
    y_preds = model.predict(X_test)
    #Make / store our prediction results in sample_pred_df
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)